In [2]:
from ranking import rank_resumes
import pandas as pd
import random
import glob

N = 20

# grab all 'resume.pdf' filepaths recursively from a output directory
resume_paths = glob.glob("output_files/**/resume.pdf", recursive=True)  # limit to first 10 for speed
#randomize and pick N
resume_paths = random.sample(resume_paths, min(N, len(resume_paths)))
job_listing_path = "input_data/Driscolls Data Services Developer.pdf"

results = rank_resumes(
    resume_paths=resume_paths,
    job_listing_path=job_listing_path,
    use_llm_job_extract=True,     # set false if you want pure-text only
    include_evidence=True,
    weights={"semantic": 0.6, "lexical": 0.3, "coverage": 0.1},
)

df = pd.DataFrame([
    {
        "resume": r.path,
        "overall": r.overall,
        "semantic": r.semantic,
        "lexical": r.lexical,
        "coverage": r.coverage,
        "matched_keywords": ", ".join((r.matched_keywords or [])[:12]),
        "missing_keywords": ", ".join((r.missing_keywords or [])[:12]),
        "evidence": " | ".join((r.evidence_snippets or [])[:2]),
    }
    for r in results
]).sort_values("overall", ascending=False)

# nice notebook table
df.style.format({"overall": "{:.3f}", "semantic": "{:.3f}", "lexical": "{:.3f}", "coverage": "{:.3f}"})

INFO - Ranking 20 resumes against job listing: input_data/Driscolls Data Services Developer.pdf
INFO - Loading job listing text
WARNING - LLM extraction requested but no LLM provided, skipping extraction
INFO - Loading resume texts
INFO - Initializing scoring strategies
INFO - Scoring 20 resumes
INFO - Scoring resume 1/20: output_files/Adobe/DataEngineerBra_20251001215030/resume.pdf
INFO - Scoring resume 2/20: output_files/ThomsonReuters/DataScienceInte_20241231205404/resume.pdf
INFO - Scoring resume 3/20: output_files/Adobe/DataScienceEngi_20241220172036/resume.pdf
INFO - Scoring resume 4/20: output_files/Intuit/StaffTechnicalD_20250624181524/resume.pdf
INFO - Scoring resume 5/20: output_files/Intuitive/DataAnalyst_20250208202240/resume.pdf
INFO - Scoring resume 6/20: output_files/Meta/DataAutomationA_20250114202028/resume.pdf
INFO - Scoring resume 7/20: output_files/EricssonInc/InternGlobalArt_20250108211111/resume.pdf
INFO - Scoring resume 8/20: output_files/CityOfSantaClara/Busines

,resume,overall,semantic,lexical,coverage,matched_keywords,missing_keywords,evidence
0,output_files/WesternDigital/DataScienceEngi_20250111211747/resume.pdf,0.483,0.557,0.418,0.233,,,"Masters in Data Science | · Designed RESTful API endpoints to automate and optimize data interactions, reducing manual data handling efforts."
1,output_files/Meta/DataAutomationA_20250114202028/resume.pdf,0.466,0.514,0.441,0.252,,,"and consolidating services. | · Optimized financial tracking through custom spreadsheet solutions, reducing reporting time."
2,output_files/ThomsonReuters/DataScienceInte_20241231205404/resume.pdf,0.465,0.505,0.454,0.255,,,· Awarded ’Best Innovation’ at the University Tech Fest 2022 for developing an AI-driven tool that analyzes and processes large | Masters in Data Science
3,output_files/Adobe/DataScienceSumm_20250120205150/resume.pdf,0.456,0.501,0.431,0.260,,,data analysis and reporting. | data analysis and reporting.
4,output_files/Databricks/DataScienceInte_20250105204321/resume.pdf,0.452,0.478,0.468,0.250,,,Databricks’ focus on advanced data science methodologies. | presenting data insights effectively.
5,output_files/AsmlUs/InternHmiCustom_20250117203226/resume.pdf,0.444,0.463,0.470,0.252,,,"Masters in Data Science | · Designed RESTful API endpoints to automate and optimize data interactions, reducing manual data handling efforts."
6,output_files/Intuitive/DataAnalyst_20250208202240/resume.pdf,0.441,0.467,0.445,0.270,,,"· Built scalable data processing pipelines handling multiple LLM APIs and response formats. | processes, enhancing data integrity and accessibility."
7,output_files/Adobe/DataScienceEngi_20241220172036/resume.pdf,0.439,0.464,0.458,0.228,,,analytics and reporting. | analytics and reporting.
8,output_files/Intuit/StaffTechnicalD_20250624181524/resume.pdf,0.433,0.444,0.469,0.260,,,"integration, improving data integrity and efficiency. | Masters in Data Science"
9,output_files/UniversityOfCaliforniaMerced/DataEngineer_20250829185019/resume.pdf,0.427,0.435,0.469,0.257,,,training and support. | performance through effective data management strategies.


In [3]:
import os
from datetime import datetime
from pathlib import Path
from parsers import load_doc_text
from job_extract import extract_job_features_from_text
from application_generator import JobApplicationBuild
from models import ChatGPT
from prompts.resume_section_prompts import RESUME_WRITER_PERSONA
from creds import OPENAI_KEY
from utils import build_application_destination

job_listing_path = "input_data/Driscolls Data Services Developer.pdf"

llm = ChatGPT(
    api_key=OPENAI_KEY,
    model="gpt-4o",
    system_prompt=RESUME_WRITER_PERSONA,
 )

job_text = load_doc_text(job_listing_path)
job_features = extract_job_features_from_text(job_text, llm)

company_name = job_features.company_name or "UnknownOrg"
job_title = job_features.job_title or Path(job_listing_path).stem
shared_timestamp = datetime.now().strftime(r"%Y%m%d%H%M%S")

ranking_run_dir = build_application_destination(
    company_name=company_name,
    job_title=job_title,
    output_destination="output_files",
    timestamp=shared_timestamp,
 )

print(f"Saving generated application to: {ranking_run_dir}")

/Users/aaronrdankert/projects/job_hunter_toolbox/application_generator.py:569: SyntaxWarning: invalid escape sequence '\B'
  block_start_string="\BLOCK{",
/Users/aaronrdankert/projects/job_hunter_toolbox/application_generator.py:571: SyntaxWarning: invalid escape sequence '\V'
  variable_start_string="\VAR{",
/Users/aaronrdankert/projects/job_hunter_toolbox/application_generator.py:573: SyntaxWarning: invalid escape sequence '\#'
  comment_start_string="\#{",


Saving generated application to: output_files/DriscollS/SeniorDataServi_20260101193937


In [ ]:

import os
from latex_toolbox import compile_resume_latex_to_pdf

tex_path = "output_files/DriscollS/SeniorDataServi_20260101193937/resume.tex"
cls_path = "templates/less_basic_template.cls"  # or your chosen .cls file
output_dir = os.path.dirname(tex_path)

success = compile_resume_latex_to_pdf(
    tex_filepath=tex_path,
    cls_filepath=cls_path,
    output_destination_path=output_dir,
)

if success:
    print(f"Resume PDF saved to {tex_path.replace('.tex', '.pdf')}")
else:
    raise RuntimeError("LaTeX compilation failed")
